In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Assuming you have loaded your ECG dataset and labels
# X, y = load_ecg_data()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base VGG-16 model
def create_vgg16(input_shape, num_classes):
    model = Sequential()

    # Block 1
    model.add(Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(2, strides=2))

    # Block 2
    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(2, strides=2))

    # Block 3
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(2, strides=2))

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create the VGG-16 model for your specific input shape and number of classes
input_shape = X_train.shape[1:]  # Shape of one ECG signal
num_classes = 5  # Replace with your actual number of classes
vgg16_model = create_vgg16(input_shape, num_classes)

# Compile the model with Adam optimizer, a learning rate of 0.01, and categorical crossentropy loss
adam_optimizer = Adam(lr=0.01)
vgg16_model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with a batch size of 128
vgg16_model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the model
accuracy = vgg16_model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")
